<a href="https://colab.research.google.com/github/oshin94/E-mail-Classification/blob/main/Main_code_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 60.2 MB/s 
     |████████████████████████████████| 7.6 MB 45.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 78.2 MB/s 
     |████████████████████████████████| 132 kB 71.1 MB/s 
     |████████████████████████████████| 127 kB 75.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 2.3 MB/s 


In [2]:
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split

from transformers import pipeline
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

from tqdm import tqdm

import tensorflow as tf

from datasets import load_dataset
import torch

In [3]:
torch.cuda.is_available()

True

# clean data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.listdir("/content/drive/MyDrive/data/cleaned/")

['1', '6', '4', '5', '3', '2', 'data.csv', 'data_balanced.csv']

In [6]:
folders = os.listdir("/content/drive/MyDrive/data/cleaned/")
print(folders)

# df = pd.DataFrame(columns = ['label','text'])

# for folder in folders:
#     #print("\n\n\nSampled")
#     sampled_files = os.listdir("/content/drive/MyDrive/data/cleaned/"+folder+"/")
#     #print(sampled_files, len(sampled_files))
#     for file in sampled_files:
#         with open("/content/drive/MyDrive/data/cleaned/"+folder+"/"+file, "r") as f:
#             data = f.read()
#             df = df.append({'label': str(int(folder)-1), 'text' : data}, ignore_index = True)
# print(df)

df = pd.read_csv("/content/drive/MyDrive/data/cleaned/data_balanced.csv")
df = df.sample(frac = 1)

['1', '6', '4', '5', '3', '2', 'data.csv', 'data_balanced.csv']


In [7]:
df

,label,text
3171,0,subject: enron mentions how important is this...
3860,1,subject: l.a. confidential has been received. ...
2168,5,subject: fw: hedging stratege for full require...
4685,5,attached are suggested comments on the draft...
3189,0,subject: feedback from kl/js ca memo perhaps ...
...,...,...
5108,0,"james.steffes@enron.com, joe.hartsoe@enron.com..."
2743,3,"subject: rosalee.fleming@enron.com, sandra.mcc..."
5110,2,subject: re: thanks. look forward to seeing y...
3718,0,subject: re: ene officer elections we should ...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [8]:
train, test = train_test_split(df, test_size=0.2)
train.to_csv("train.csv", index=0)
test.to_csv("test.csv", index=0)

In [9]:
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-598be1e03982eb84/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
dataset["test"][10]

{'label': 2,
 'text': "subject: re: eeegads...  hey, when are you scheduled to go in for your attitude labotomy?    paul kaufman/enron@enronxgate 05/10/2001 01:51 pm  cc: subject: re: eeegads...  i'll be there, as will sandi mccubbin. are you interested in going? i thought not.  -----original message----- sent: thursday, may 10, 2001 11:02 am subject: eeegads...  don't know how i got on the list, but just got my invitation to the western conference of pucs meeting june 3-6 where the guest speakers are frank wolak and gary locke. oh dear....  best, jeff "}

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

# Training

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(output_dir="test_trainer")

In [14]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/data/test_trainer/", evaluation_strategy="epoch", num_train_epochs=10, per_device_train_batch_size=18, per_device_eval_batch_size=12)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4236
  Num Epochs = 10
  Instantaneous batch size per device = 18
  Total train batch size (w. parallel, distributed & accumulation) = 18
  Gradient Accumulation steps = 1
  Total optimization steps = 2360
  Number of trainable parameters = 109486854


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.374320,0.885849
2,No log,0.195753,0.944340
3,0.594400,0.229507,0.941509
4,0.594400,0.186385,0.953774
5,0.091200,0.193273,0.956604
6,0.091200,0.159387,0.959434
7,0.042100,0.214659,0.957547
8,0.042100,0.158619,0.964151
9,0.028900,0.181598,0.961321
10,0.028900,0.185037,0.960377


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1060
  Batch size = 12
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1060
  Batch size = 12
Saving model checkpoint to /content/drive/MyDrive/data/test_trainer/checkpoint-500
Configuration saved in /content/drive/MyDrive/data/test_trainer/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/data/test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding 

TrainOutput(global_step=2360, training_loss=0.16366182969788373, metrics={'train_runtime': 4253.8013, 'train_samples_per_second': 9.958, 'train_steps_per_second': 0.555, 'total_flos': 1.114578458468352e+16, 'train_loss': 0.16366182969788373, 'epoch': 10.0})